In [ ]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/12500/1375107/compressed/train.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1625512696&Signature=jFkWpFWryXz4VptMmfpAhYW2PBmgJP49GjzSLrrRdu9eLBo3WBHQLB3tCyfVsPd%2BelpOt1i8e%2Bl2yrykxOlW3IBHwSj7UTr7a%2BnPl7AFdPkHNPFGnd6T6sWrIhaa5pauTRUOJqHy5Wz4%2FIbsw8EiuaAfKrNjDP8Jysy0pn52MxNBnGTPReXPBRr1iFsW6jID2iw2juYftRK1WYxAC0N6Qx2alMQlNe7FdY6MIq%2BekrEnWAlbPvyHZ82eoCUkGTMt%2B7yolvTjAzsTGc0%2FoBIAMx7m91rH6T0fcW1zqgQGUaa%2F5LR5flyNUNgJB%2B07%2BfEYrgzmB7TBEeuIPVpiWc1vFg%3D%3D&response-content-disposition=attachment%3B+filename%3Dtrain.csv.zip" -c -O 'train.csv.zip'
!pip install transformers

--2021-07-02 19:18:36--  https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/12500/1375107/compressed/train.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1625512696&Signature=jFkWpFWryXz4VptMmfpAhYW2PBmgJP49GjzSLrrRdu9eLBo3WBHQLB3tCyfVsPd%2BelpOt1i8e%2Bl2yrykxOlW3IBHwSj7UTr7a%2BnPl7AFdPkHNPFGnd6T6sWrIhaa5pauTRUOJqHy5Wz4%2FIbsw8EiuaAfKrNjDP8Jysy0pn52MxNBnGTPReXPBRr1iFsW6jID2iw2juYftRK1WYxAC0N6Qx2alMQlNe7FdY6MIq%2BekrEnWAlbPvyHZ82eoCUkGTMt%2B7yolvTjAzsTGc0%2FoBIAMx7m91rH6T0fcW1zqgQGUaa%2F5LR5flyNUNgJB%2B07%2BfEYrgzmB7TBEeuIPVpiWc1vFg%3D%3D&response-content-disposition=attachment%3B+filename%3Dtrain.csv.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 74.125.197.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 289820185 (276M) [application/zip]
Saving to: ‘train.csv.zip’

train.csv.zip    

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
import tensorflow as tf
import io
import pickle
 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
from tensorflow.keras.layers import Conv1D,MaxPooling1D,Flatten,concatenate,Average
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import LSTM,Bidirectional,Dense,LeakyReLU,BatchNormalization,Embedding,Input,Dropout,SpatialDropout1D,GlobalAveragePooling1D,GlobalMaxPooling1D,Concatenate,Add
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping
##Source: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html
from sklearn.metrics import roc_curve,auc,f1_score,confusion_matrix
from transformers import AutoTokenizer,TFBertModel
 
import nltk
nltk.download('all')
 
##Source: https://stackoverflow.com/a/14463362/8211103
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

In [ ]:
def plot_fpr_tpr(y_train,y_pred,ax1,name):
    fpr, tpr, threshold = roc_curve(y_train,y_pred)
    auc_score = round(auc(fpr,tpr),3)
    ax1.plot(fpr,tpr,label=f"{name} Auc-Score: {auc_score}")
    ax1.grid()
    ax1.set_xlabel('FPR')
    ax1.set_ylabel('TPR')
    ax1.title.set_text('')
    print(f"{name} Auc-Score: {auc_score}")
    return threshold[np.argmax(tpr*(1-fpr))]
 
def plot(y_train,y_pred,y_test,y_test_pred):
  fig,(ax1,ax2,ax3) = plt.subplots(1,3,figsize=(18,5))
  threshold = plot_fpr_tpr(y_train,y_pred,ax1,'Train')
  _ = plot_fpr_tpr(y_test,y_test_pred,ax1,'Test')
  ax1.legend()
  
  y_pred = (y_pred>threshold).astype(int)
  y_test_pred = (y_test_pred>threshold).astype(int)
 
  cm = confusion_matrix(np.array(y_train),y_pred)
  sns.heatmap(cm,ax=ax2,fmt='6d',annot=True)
  ax2.title.set_text('Train Confution Metrics')
  ax2.set_ylabel('Actual label')
  ax2.set_xlabel('Predicted label')
 
  cm = confusion_matrix(np.array(y_test),y_test_pred)
  sns.heatmap(cm,ax=ax3,fmt='6d',annot=True)
  ax3.title.set_text('Test Confution Metrics')
  ax3.set_ylabel('Actual label')
  ax3.set_xlabel('Predicted label')
 
  ## We will use AUC's threshold for f1 score
  ## Here we will try to get high f1-score for our model
  f11 = round(f1_score(y_train,y_pred),3)
  f12 = round(f1_score(y_test,y_test_pred),3)
  print(f"Train F1-Score: {f11}, Test F1-Score: {f12},")
  return f11,f12
 
 
 
 
def f1_with_auc(y_train,y_pred):
  y_train = y_train[:,1]
  y_pred = y_pred[:,1]
 
  y_train = y_train.numpy()
  y_pred = y_pred.numpy()
 
  fpr, tpr, threshold = roc_curve(y_train,y_pred)
  threshold = threshold[np.argmax(tpr*(1-fpr))]
  y_train = (y_train>threshold).astype(int)
  y_pred = (y_pred>threshold).astype(int)
  return f1_score(y_train,y_pred)
 
 
 
 
 
 
symbols_to_isolate = '.,?!-;*"…:—()%#$&_/@＼・ω+=”“[]^–>\\°<~•≠™ˈʊɒ∞§{}·τα❤☺ɡ|¢→̶`❥━┣┫┗Ｏ►★©―ɪ✔®\x96\x92●£♥➤´¹☕≈÷♡◐║▬′ɔː€۩۞†μ✒➥═☆ˌ◄½ʻπδηλσερνʃ✬ＳＵＰＥＲＩＴ☻±♍µº¾✓◾؟．⬅℅»Вав❣⋅¿¬♫ＣＭβ█▓▒░⇒⭐›¡₂₃❧▰▔◞▀▂▃▄▅▆▇↙γ̄″☹➡«φ⅓„✋：¥̲̅́∙‛◇✏▷❓❗¶˚˙）сиʿ✨。ɑ\x80◕！％¯−ﬂﬁ₁²ʌ¼⁴⁄₄⌠♭✘╪▶☭✭♪☔☠♂☃☎✈✌✰❆☙○‣⚓年∎ℒ▪▙☏⅛ｃａｓǀ℮¸ｗ‚∼‖ℳ❄←☼⋆ʒ⊂、⅔¨͡๏⚾⚽Φ×θ￦？（℃⏩☮⚠月✊❌⭕▸■⇌☐☑⚡☄ǫ╭∩╮，例＞ʕɐ̣Δ₀✞┈╱╲▏▕┃╰▊▋╯┳┊≥☒↑☝ɹ✅☛♩☞ＡＪＢ◔◡↓♀⬆̱ℏ\x91⠀ˤ╚↺⇤∏✾◦♬³の｜／∵∴√Ω¤☜▲↳▫‿⬇✧ｏｖｍ－２０８＇‰≤∕ˆ⚜☁'
symbols_to_delete = '\n🍕\r🐵😑\xa0\ue014\t\uf818\uf04a\xad😢🐶️\uf0e0😜😎👊\u200b\u200e😁عدويهصقأناخلىبمغر😍💖💵Е👎😀😂\u202a\u202c🔥😄🏻💥ᴍʏʀᴇɴᴅᴏᴀᴋʜᴜʟᴛᴄᴘʙғᴊᴡɢ😋👏שלוםבי😱‼\x81エンジ故障\u2009🚌ᴵ͞🌟😊😳😧🙀😐😕\u200f👍😮😃😘אעכח💩💯⛽🚄🏼ஜ😖ᴠ🚲‐😟😈💪🙏🎯🌹😇💔😡\x7f👌ἐὶήιὲκἀίῃἴξ🙄Ｈ😠\ufeff\u2028😉😤⛺🙂\u3000تحكسة👮💙فزط😏🍾🎉😞\u2008🏾😅😭👻😥😔😓🏽🎆🍻🍽🎶🌺🤔😪\x08‑🐰🐇🐱🙆😨🙃💕𝘊𝘦𝘳𝘢𝘵𝘰𝘤𝘺𝘴𝘪𝘧𝘮𝘣💗💚地獄谷улкнПоАН🐾🐕😆ה🔗🚽歌舞伎🙈😴🏿🤗🇺🇸мυтѕ⤵🏆🎃😩\u200a🌠🐟💫💰💎эпрд\x95🖐🙅⛲🍰🤐👆🙌\u2002💛🙁👀🙊🙉\u2004ˢᵒʳʸᴼᴷᴺʷᵗʰᵉᵘ\x13🚬🤓\ue602😵άοόςέὸתמדףנרךצט😒͝🆕👅👥👄🔄🔤👉👤👶👲🔛🎓\uf0b7\uf04c\x9f\x10成都😣⏺😌🤑🌏😯ех😲Ἰᾶὁ💞🚓🔔📚🏀👐\u202d💤🍇\ue613小土豆🏡❔⁉\u202f👠》कर्मा🇹🇼🌸蔡英文🌞🎲レクサス😛外国人关系Сб💋💀🎄💜🤢َِьыгя不是\x9c\x9d🗑\u2005💃📣👿༼つ༽😰ḷЗз▱ц￼🤣卖温哥华议会下降你失去所有的钱加拿大坏税骗子🐝ツ🎅\x85🍺آإشء🎵🌎͟ἔ油别克🤡🤥😬🤧й\u2003🚀🤴ʲшчИОРФДЯМюж😝🖑ὐύύ特殊作戦群щ💨圆明园קℐ🏈😺🌍⏏ệ🍔🐮🍁🍆🍑🌮🌯🤦\u200d𝓒𝓲𝓿𝓵안영하세요ЖљКћ🍀😫🤤ῦ我出生在了可以说普通话汉语好极🎼🕺🍸🥂🗽🎇🎊🆘🤠👩🖒🚪天一家⚲\u2006⚭⚆⬭⬯⏖新✀╌🇫🇷🇩🇪🇮🇬🇧😷🇨🇦ХШ🌐\x1f杀鸡给猴看ʁ𝗪𝗵𝗲𝗻𝘆𝗼𝘂𝗿𝗮𝗹𝗶𝘇𝗯𝘁𝗰𝘀𝘅𝗽𝘄𝗱📺ϖ\u2000үսᴦᎥһͺ\u2007հ\u2001ɩｙｅ൦ｌƽｈ𝐓𝐡𝐞𝐫𝐮𝐝𝐚𝐃𝐜𝐩𝐭𝐢𝐨𝐧Ƅᴨןᑯ໐ΤᏧ௦Іᴑ܁𝐬𝐰𝐲𝐛𝐦𝐯𝐑𝐙𝐣𝐇𝐂𝐘𝟎ԜТᗞ౦〔Ꭻ𝐳𝐔𝐱𝟔𝟓𝐅🐋ﬃ💘💓ё𝘥𝘯𝘶💐🌋🌄🌅𝙬𝙖𝙨𝙤𝙣𝙡𝙮𝙘𝙠𝙚𝙙𝙜𝙧𝙥𝙩𝙪𝙗𝙞𝙝𝙛👺🐷ℋ𝐀𝐥𝐪🚶𝙢Ἱ🤘ͦ💸ج패티Ｗ𝙇ᵻ👂👃ɜ🎫\uf0a7БУі🚢🚂ગુજરાતીῆ🏃𝓬𝓻𝓴𝓮𝓽𝓼☘﴾̯﴿₽\ue807𝑻𝒆𝒍𝒕𝒉𝒓𝒖𝒂𝒏𝒅𝒔𝒎𝒗𝒊👽😙\u200cЛ‒🎾👹⎌🏒⛸公寓养宠物吗🏄🐀🚑🤷操美𝒑𝒚𝒐𝑴🤙🐒欢迎来到阿拉斯ספ𝙫🐈𝒌𝙊𝙭𝙆𝙋𝙍𝘼𝙅ﷻ🦄巨收赢得白鬼愤怒要买额ẽ🚗🐳𝟏𝐟𝟖𝟑𝟕𝒄𝟗𝐠𝙄𝙃👇锟斤拷𝗢𝟳𝟱𝟬⦁マルハニチロ株式社⛷한국어ㄸㅓ니͜ʖ𝘿𝙔₵𝒩ℯ𝒾𝓁𝒶𝓉𝓇𝓊𝓃𝓈𝓅ℴ𝒻𝒽𝓀𝓌𝒸𝓎𝙏ζ𝙟𝘃𝗺𝟮𝟭𝟯𝟲👋🦊多伦🐽🎻🎹⛓🏹🍷🦆为和中友谊祝贺与其想象对法如直接问用自己猜本传教士没积唯认识基督徒曾经让相信耶稣复活死怪他但当们聊些政治题时候战胜因圣把全堂结婚孩恐惧且栗谓这样还♾🎸🤕🤒⛑🎁批判检讨🏝🦁🙋😶쥐스탱트뤼도석유가격인상이경제황을렵게만들지않록잘관리해야합다캐나에서대마초와화약금의품런성분갈때는반드시허된사용🔫👁凸ὰ💲🗯𝙈Ἄ𝒇𝒈𝒘𝒃𝑬𝑶𝕾𝖙𝖗𝖆𝖎𝖌𝖍𝖕𝖊𝖔𝖑𝖉𝖓𝖐𝖜𝖞𝖚𝖇𝕿𝖘𝖄𝖛𝖒𝖋𝖂𝕴𝖟𝖈𝕸👑🚿💡知彼百\uf005𝙀𝒛𝑲𝑳𝑾𝒋𝟒😦𝙒𝘾𝘽🏐𝘩𝘨ὼṑ𝑱𝑹𝑫𝑵𝑪🇰🇵👾ᓇᒧᔭᐃᐧᐦᑳᐨᓃᓂᑲᐸᑭᑎᓀᐣ🐄🎈🔨🐎🤞🐸💟🎰🌝🛳点击查版🍭𝑥𝑦𝑧ＮＧ👣\uf020っ🏉ф💭🎥Ξ🐴👨🤳🦍\x0b🍩𝑯𝒒😗𝟐🏂👳🍗🕉🐲چی𝑮𝗕𝗴🍒ꜥⲣⲏ🐑⏰鉄リ事件ї💊「」\uf203\uf09a\uf222\ue608\uf202\uf099\uf469\ue607\uf410\ue600燻製シ虚偽屁理屈Г𝑩𝑰𝒀𝑺🌤𝗳𝗜𝗙𝗦𝗧🍊ὺἈἡχῖΛ⤏🇳𝒙ψՁմեռայինրւդձ冬至ὀ𝒁🔹🤚🍎𝑷🐂💅𝘬𝘱𝘸𝘷𝘐𝘭𝘓𝘖𝘹𝘲𝘫کΒώ💢ΜΟΝΑΕ🇱♲𝝈↴💒⊘Ȼ🚴🖕🖤🥘📍👈➕🚫🎨🌑🐻𝐎𝐍𝐊𝑭🤖🎎😼🕷ｇｒｎｔｉｄｕｆｂｋ𝟰🇴🇭🇻🇲𝗞𝗭𝗘𝗤👼📉🍟🍦🌈🔭《🐊🐍\uf10aლڡ🐦\U0001f92f\U0001f92a🐡💳ἱ🙇𝗸𝗟𝗠𝗷🥜さようなら🔼'
 
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~`" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
small_caps_mapping = { 
    "ᴀ": "a", "ʙ": "b", "ᴄ": "c", "ᴅ": "d", "ᴇ": "e", "ғ": "f", "ɢ": "g", "ʜ": "h", "ɪ": "i", 
    "ᴊ": "j", "ᴋ": "k", "ʟ": "l", "ᴍ": "m", "ɴ": "n", "ᴏ": "o", "ᴘ": "p", "ǫ": "q", "ʀ": "r", 
    "s": "s", "ᴛ": "t", "ᴜ": "u", "ᴠ": "v", "ᴡ": "w", "x": "x", "ʏ": "y", "ᴢ": "z"}
contraction_mapping = {
    "ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", 
    "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", 
    "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  
    "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": 
    "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", 
    "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", 
    "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", 
    "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", 
    "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", 
    "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", 
    "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's":"this is","that'd": "that would", 
    "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", 
    "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", 
    "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", 
    "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", 
    "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", 
    "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have",
    "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", 
    "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", 
    "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
    "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have",
    "trump's": "trump is", "obama's": "obama is", "canada's": "canada is", "today's": "today is"}
specail_signs = { "…": "...", "₂": "2"}
specials = ["’", "‘", "´", "`"]
 
from nltk.tokenize.treebank import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()
 
 
isolate_dict = {ord(c):f' {c} ' for c in symbols_to_isolate}
remove_dict = {ord(c):f'' for c in symbols_to_delete}
 
 
def handle_punctuation(x):
    x = x.translate(remove_dict)
    x = x.translate(isolate_dict)
    return x
 
def handle_contractions(x):
    x = tokenizer.tokenize(x)
    return x
 
def fix_quote(x):
    x = [x_[1:] if x_.startswith("'") else x_ for x_ in x]
    x = ' '.join(x)
    return x
 
def preprocess(x):
    x = handle_punctuation(x)
    x = handle_contractions(x)
    x = fix_quote(x)
    return x

In [ ]:
data = pd.read_csv('/content/train.csv.zip')
data['proba'] = data['target']
data = data[['comment_text','target','severe_toxicity','obscene','threat','insult','identity_attack','proba']]
data = data.sample(frac=1).reset_index(drop=True)
data['comment_text'] = data['comment_text'].apply(preprocess)

In [ ]:
data['target'] = (data['target']>=.5).astype(float)
 
y = data[['target','proba','proba','proba','severe_toxicity','obscene','threat','insult','identity_attack']] 
x_train,x_test,y_train,y_test = train_test_split(data['comment_text'],y,stratify=y['target'],test_size=0.1)
 
x_train = x_train.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)
y_train = y_train.values
y_test = y_test.values

In [ ]:
MAX_LEN = 237 + 2

In [ ]:
def get_sequence(txt, max_len,tokenizer):
    data = tokenizer(
        txt.tolist() ,
        return_attention_mask=False,
        return_token_type_ids=False,
        max_length=max_len,
        truncation=True,
        padding='max_length'
    )['input_ids']
    return np.array(data)

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [ ]:
x_train = get_sequence(x_train, MAX_LEN,tokenizer)
x_test = get_sequence(x_test, MAX_LEN,tokenizer)

In [ ]:
train_df = y_train.copy()
weights = np.ones((len(train_df),)) / 4
 
# Subgroup
weights += (train_df[:,1:]>=0.5).sum(axis=1).astype(bool).astype(np.int) / 4
 
# Background Positive, Subgroup Negative
weights += (( (train_df[:,0]>=0.5).astype(bool).astype(np.int) + (train_df[:,1:]<0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) / 4
 
# Background Negative, Subgroup Positive
weights += (( (train_df[:,0]<0.5).astype(bool).astype(np.int) + (train_df[:,1:]>=0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) / 4
 
# for later normalization the loss
loss_weight = 1.0 / weights.mean()
 
y_train = np.hstack([y_train,weights.reshape(-1,1)])
y_train.shape

(1624386, 10)

In [ ]:
class Dataloder(tf.keras.utils.Sequence):    
    def __init__(self, data,y, batch_size=1):
        self.data = data
        self.y = y
        self.indexes = np.arange(len(self.data))
        self.batch_size = batch_size
 
 
    def __getitem__(self, i):
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
 
        data = self.data[start:stop,:]
        y = self.y[start:stop,:]
        attention = (data>0).astype(int)
 
        return tuple([[data,attention],y])
 
    def __len__(self):
        return len(self.indexes) // self.batch_size

In [ ]:
BATCH_SIZE = 32
 
train_dataloader = Dataloder(x_train,y_train, batch_size=BATCH_SIZE)
test_dataloader = Dataloder(x_test,y_test, batch_size=BATCH_SIZE)

In [ ]:
def create_model():
    input_ids = Input((MAX_LEN,),dtype=tf.int32)
    attention_mask = Input((MAX_LEN,),dtype=tf.int32)
 
 
    bert_out = bert(input_ids=input_ids, attention_mask=attention_mask)[1]
    x = Dense(128,activation='relu')(bert_out)
    output2 = Dense(8)(x)
    x = Dropout(0.1)(x)
    output1 = Dense(1)(x)
    output = tf.keras.layers.concatenate([output1,output2])
 
    model = Model([input_ids,attention_mask],output)
    model.layers[3].trainable=True
 
    return model

In [ ]:
bert = TFBertModel.from_pretrained('bert-base-cased')
model = create_model()
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-05,epsilon=1e-08,decay=0.01,clipnorm=1.0) 
 
def in_loss(y_true,y_pred):
    xent = tf.keras.losses.BinaryCrossentropy(from_logits=True,reduction=tf.keras.losses.Reduction.NONE)
    loss1 = tf.reduce_mean(xent(y_true[:,0],y_pred[:,0]) * y_true[:,-1])
    loss2 = xent(y_true[:,1:-1],y_pred[:,1:])
    return ((loss1 * loss_weight)*0.60 + loss2*0.40)*2
 
model.load_weights("/content/drive/MyDrive/Colab Notebooks/AAIC/Case Study 1/BERT-linear-cust-loss-v2-12epoch.h5")
model.compile(optimizer=optimizer,loss=in_loss)
 
model.fit(train_dataloader,epochs=2)
model.save_weights('/content/drive/MyDrive/Colab Notebooks/AAIC/Case Study 1/BERT-linear-cust-loss-v2-14epoch.h5', overwrite=True)

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

Kaggle Submission:
* Score: 0.93063
* 1747 in TOP in TOP 56% 

![](https://drive.google.com/uc?export=view&id=1BreTSrmfZJ4zSvUxbRCvv0Ccj_OYV11M)

###Reference:
* [keras.io](https://keras.io)
* [tensorflow.org](https://tensorflow.org)
* https://huggingface.co/
* https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification
* https://github.com/jiaruxu233/Jigsaw-Unintended-Bias-in-Toxicity-Classification/blob/master/Harness_the_Beast.ipynb
* https://www.kaggle.com/sandeepkumar121995/keras-bi-gru-lstm-attention-fasttext
* https://github.com/minimaxir/textgenrnn/blob/master/textgenrnn/AttentionWeightedAverage.py
* https://www.researchgate.net/publication/341238064_Avoiding_Unintended_Bias_in_Toxicity_Classification_with_Neural_Networks
* https://www.theseus.fi/bitstream/handle/10024/226938/Quan_Do.pdf
* https://fasttext.cc/
* https://nlp.stanford.edu/projects/glove/
* https://www.nltk.org/